!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!This script contains maskign code before standardisation. however standardisation of inputs and targets was performed before the interpoléated baselines were generated via CDO for bilinear and bicubic and then masked so that they only retain the Swiss Domain

Workflow : Bilinear interpolation, bicubic interpolation, Masking to retain only the swiss domain, splitting into train, test val, gridded RMSE R2, pooled RMSE, R2

In [1]:
from dependencies import *
import sys
sys.path.append("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/Scripts/Functions")

In [ ]:
from dask.distributed import Client
client= Client()
print(client)

<Client: 'tcp://127.0.0.1:34705' processes=4 threads=4, memory=503.49 GiB>


2025-05-09 11:49:01,793 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:41593' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {('shuffle-split-13d6dd49173a41f76cbc315063e30090', 6054), ('shuffle-split-13d6dd49173a41f76cbc315063e30090', 9172), ('shuffle-split-13d6dd49173a41f76cbc315063e30090', 541), ('shuffle-split-13d6dd49173a41f76cbc315063e30090', 7860), ('shuffle-split-13d6dd49173a41f76cbc315063e30090', 5645), ('shuffle-split-13d6dd49173a41f76cbc315063e30090', 6548), ('shuffle-split-13d6dd49173a41f76cbc315063e30090', 4104), ('shuffle-split-13d6dd49173a41f76cbc315063e30090', 2383), ('shuffle-split-13d6dd49173a41f76cbc315063e30090', 5501), ('shuffle-split-13d6dd49173a41f76cbc315063e30090', 1071), ('shuffle-split-13d6dd49173a41f76cbc315063e30090', 3057), ('shuffle-split-13d6dd49173a41f76cbc315063e30090', 3960), ('shuffle-split-13d6dd49173a41f76cbc315063e30090', 662), ('shuffle-split-13d6dd49173a41f76cbc315063e30090', 7752), 

bilinear interpolation to the 1 km grid was performed using CDO remapbil with RhiresD/TabsD as reference files. Same for remapbic (Bicubic interpolation) Now they will be masked in this script so that it only retains the Swiss domain and not entire Europe

#For precipitation mask

In [3]:
hr_path = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/raw/RhiresD_1971_2022.nc"
mask_output_path = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/RhiresD_mask.nc"

ds_hr = xr.open_dataset(hr_path)

varname = "RhiresD" 

mask_rhiresd = xr.where(~np.isnan(ds_hr[varname].isel(time=0)), 1, 0)

mask_rhiresd = mask_rhiresd.squeeze(drop=True)

mask_rhiresd.to_netcdf(mask_output_path)

print(f"Mask saved at {mask_output_path}")


Mask saved at /work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/RhiresD_mask.nc


#For temperature mask!

In [4]:
hr_path = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/raw/TabsD_1971_2022.nc"
mask_output_path = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/TabsD_mask.nc"

ds_hr = xr.open_dataset(hr_path)

varname = "TabsD" 

mask_tabsd = xr.where(~np.isnan(ds_hr[varname].isel(time=0)), 1, 0)

mask_tabsd = mask_tabsd.squeeze(drop=True)



In [5]:
mask_tabsd.to_netcdf(mask_output_path)

print(f"Mask saved at {mask_output_path}")

Mask saved at /work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/TabsD_mask.nc


#Application of the mask to the two datasets : RhiresD and TabsD masks (1km) to the bicubically interpolated 1 km datasets features from Sven`s files so that it only retains the Swiss Domain


Lazy loading was used below to allow for parallel computation. The data was not loading due to xarray loading everything at once and the kernel was crashing

In [8]:
bicubic_ds_precip = xr.open_dataset("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processing/features_precip_bicubic.nc",chunks={"time": 500})

# Masking the features to retain the swiss domain
masked_bicubic_rhiresd = bicubic_ds_precip * mask_rhiresd

masked_bicubic_rhiresd.to_netcdf("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/features_precip_masked_bicubic.nc",
                                 engine="netcdf4",
                                    compute=True)

In [12]:
bicubic_ds_temp = xr.open_dataset("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processing/features_tas_bicubic.nc",chunks={"time": 500})

# Masking the features to retain the swiss domain
masked_bicubic_temp = bicubic_ds_temp * mask_tabsd

masked_bicubic_temp.to_netcdf("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/features_tas_masked_bicubic.nc",
                                 engine="netcdf4",
                                    compute=True)

For reducing computation time
#Could have used h5netcdf for faster computation
#Could have asked for more resources installed dask distributed, and used all 4 CPU cores, computing time reduced to 30 seconds!!!

Bilinear interpolated outputs : Maskign with the 1 km RhiresD and TabsD mask

In [13]:
bilinear_ds_precip = xr.open_dataset("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processing/features_precip_biliear.nc",chunks={"time": 500})

# Masking
masked_bilinear_precip = bilinear_ds_precip * mask_rhiresd

masked_bilinear_precip.to_netcdf("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bilinear/features_precip_masked_bilinear.nc",
                                 engine="netcdf4",
                                    compute=True)

In [14]:
bilinear_ds_temp = xr.open_dataset("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processing/features_tas_biliear.nc",chunks={"time": 500})

# Masking
masked_bilinear_temp = bilinear_ds_temp * mask_tabsd

masked_bilinear_temp.to_netcdf("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bilinear/features_tas_masked_bilinear.nc",
                                 engine="netcdf4",
                                    compute=True)

xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxCHECKING SHAPExxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

In [9]:

features_ds_precip = xr.open_dataset("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/features_precip_masked_bicubic.nc")["pr"]

In [16]:
features_ds_temp= xr.open_dataset("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/features_tas_masked_bicubic.nc")["tas"]

In [11]:
features_ds_precip.shape

(18993, 265, 370)

In [18]:
features_ds_temp.shape

(18993, 265, 370)

In [12]:
targets_precip= xr.open_dataset("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/raw/RhiresD_1971_2022.nc")["RhiresD"]
targets_precip.shape

(18993, 265, 370)

In [20]:
targets_temp= xr.open_dataset("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/raw/TabsD_1971_2022.nc")["TabsD"]
targets_temp.shape

(18993, 265, 370)

SHAPES MATCH!!!!!!

The entire dataset will be divided into tain, test val for both bilinear and bicubic interpolation for calculating standardisation metric etc


Split will be performed on one dataset, then replicated for the other


In [13]:
import sys
sys.path.append("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/ML_models")

from Train_Test_Val import split_by_decade


In [14]:
precip_path = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Combined/features_precip_masked_bicubic.nc"
tas_path = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Combined/features_tas_masked_bicubic.nc"

output_base = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic"
ds_precip = xr.open_dataset(precip_path, chunks={"time": 50})
ds_tas = xr.open_dataset(tas_path, chunks={"time": 50})

times = ds_precip['time'].values  # Assumes both datasets have matching time axis, one of the time coordinates from both datasets extracted


In [15]:
train_idx, val_idx, test_idx = split_by_decade(times, seed=42)  # See has been fixed for reproducibility

In [16]:
import os

In [17]:
train_idx, val_idx, test_idx

(tensor([ 2751,  2462,  2313,  ..., 18657, 18129, 18958]),
 tensor([  486,  1772,  1052,  ..., 18762, 18652, 18403]),
 tensor([ 2936,   400,  3011,  ..., 18911, 18736, 18547]))

In [18]:
print(train_idx.shape)
print(val_idx.shape)
print(test_idx.shape)

torch.Size([13293])
torch.Size([3796])
torch.Size([1904])


In [19]:
def save_splits(ds, indices, subset_folder, filename):
    subset = ds.isel(time=indices)
    subset = subset.chunk({"time": 50}) 
    save_dir = os.path.join(output_base, subset_folder)
    os.makedirs(save_dir, exist_ok=True)
    save_path = os.path.join(save_dir, filename)
    
    delayed_obj = subset.to_netcdf(save_path, engine="netcdf4", compute=True)
    
    print(f"Saved {filename}")


In [20]:
# Precipitation input dataset for bicubic interpolation split into three categories
save_splits(ds_precip, train_idx, "Train", "features_precip_masked_bicubic_train.nc")
save_splits(ds_precip, val_idx, "Val", "features_precip_masked_bicubic_val.nc")
save_splits(ds_precip, test_idx, "Test", "features_precip_masked_bicubic_test.nc")

Saved features_precip_masked_bicubic_train.nc
Saved features_precip_masked_bicubic_val.nc
Saved features_precip_masked_bicubic_test.nc


In [21]:
# Temperature input dataset for bicubic interpolation split into three categories
save_splits(ds_tas, train_idx, "Train", "features_tas_masked_bicubic_train.nc")
save_splits(ds_tas, val_idx, "Val", "features_tas_masked_bicubic_val.nc")
save_splits(ds_tas, test_idx, "Test", "features_tas_masked_bicubic_test.nc")

2025-05-09 11:49:01,789 - distributed.worker - ERROR - failed during get data with tcp://127.0.0.1:35323 -> tcp://127.0.0.1:41593
Traceback (most recent call last):
  File "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/MyPythonEnvironment/lib/python3.12/site-packages/tornado/iostream.py", line 861, in _read_to_buffer
    bytes_read = self.read_from_fd(buf)
                 ^^^^^^^^^^^^^^^^^^^^^^
  File "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/MyPythonEnvironment/lib/python3.12/site-packages/tornado/iostream.py", line 1116, in read_from_fd
    return self.socket.recv_into(buf, len(buf))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
ConnectionResetError: [Errno 104] Connection reset by peer

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/MyPythonEnvironment/lib/python3.12/site-packages/distributed/worker.py", line 1795, in get_data
    response = await co

Saved features_tas_masked_bicubic_train.nc
Saved features_tas_masked_bicubic_val.nc
Saved features_tas_masked_bicubic_test.nc


Also splittign the targets along the very same lines usign the reproducible split

In [35]:
precip_target = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/raw/RhiresD_1971_2022.nc"
tas_target = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/raw/TabsD_1971_2022.nc"

output_base = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic"
ds_precip = xr.open_dataset(precip_target, chunks={"time": 50})
ds_tas = xr.open_dataset(tas_target, chunks={"time": 50})

times = ds_precip['time'].values  # Assumes both datasets have matching time axis, one of the time coordinates from both datasets extracted


In [36]:
# Precipitation input dataset for bicubic interpolation split into three categories
save_splits(ds_precip, train_idx, "Train", "targets_precip_masked_train.nc")
save_splits(ds_precip, val_idx, "Val", "targets_precip_masked_val.nc")
save_splits(ds_precip, test_idx, "Test", "targets_precip_masked_test.nc")

Saved targets_precip_masked_train.nc
Saved targets_precip_masked_val.nc
Saved targets_precip_masked_test.nc


In [37]:
# Precipitation input dataset for bicubic interpolation split into three categories
save_splits(ds_tas, train_idx, "Train", "targets_tas_masked_train.nc")
save_splits(ds_tas, val_idx, "Val", "targets_tas_masked_val.nc")
save_splits(ds_tas, test_idx, "Test", "targets_tas_masked_test.nc")

Saved targets_tas_masked_train.nc
Saved targets_tas_masked_val.nc
Saved targets_tas_masked_test.nc


xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxDoing the same train test val split for bilinear nterpolation xxxxxxxxx

In [15]:
precip_path_bilinear = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Combined/features_precip_masked_bilinear.nc"
tas_path_bilinear = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Combined/features_tas_masked_bilinear.nc"

output_base = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bilinear"
ds_precip_bilinear = xr.open_dataset(precip_path_bilinear, chunks={"time": 50})
ds_tas_bilinear = xr.open_dataset(tas_path_bilinear, chunks={"time": 50})

times = ds_precip_bilinear['time'].values  # Assumes both datasets have matching time axis, one of the time coordinates from both datasets extracted

In [41]:
# Precipitation input dataset for bicubic interpolation split into three categories
save_splits(ds_precip_bilinear, train_idx, "Train", "features_precip_masked_bilinear_train.nc")
save_splits(ds_precip_bilinear, val_idx, "Val", "features_precip_masked_bilinear_val.nc")
save_splits(ds_precip_bilinear, test_idx, "Test", "features_precip_masked_bilinear_test.nc")

2025-05-01 17:16:50,663 - distributed.worker - ERROR - failed during get data with tcp://127.0.0.1:38899 -> tcp://127.0.0.1:46187
Traceback (most recent call last):
  File "/users/sasthana/.local/lib/python3.12/site-packages/tornado/iostream.py", line 861, in _read_to_buffer
    bytes_read = self.read_from_fd(buf)
                 ^^^^^^^^^^^^^^^^^^^^^^
  File "/users/sasthana/.local/lib/python3.12/site-packages/tornado/iostream.py", line 1116, in read_from_fd
    return self.socket.recv_into(buf, len(buf))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
ConnectionResetError: [Errno 104] Connection reset by peer

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/users/sasthana/.local/lib/python3.12/site-packages/distributed/worker.py", line 1795, in get_data
    response = await comm.read(deserializers=serializers)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/users/sasthana/.local/lib/python3.12/site-

Saved features_precip_masked_bilinear_train.nc
Saved features_precip_masked_bilinear_val.nc
Saved features_precip_masked_bilinear_test.nc


In [16]:
# Precipitation input dataset for bicubic interpolation split into three categories
save_splits(ds_tas_bilinear, train_idx, "Train", "features_tas_masked_bilinear_train.nc")
save_splits(ds_tas_bilinear, val_idx, "Val", "features_tas_masked_bilinear_val.nc")
save_splits(ds_tas_bilinear, test_idx, "Test", "features_tas_masked_bilinear_test.nc")

Saved features_tas_masked_bilinear_train.nc
Saved features_tas_masked_bilinear_val.nc
Saved features_tas_masked_bilinear_test.nc


                                                  xxxxxxxxxxStandardisation code  xxxxxxxxxxxxxxxxxxxxxxxxxx

Standardisation has to be performed avoiding data leakage. Hence, the train -test contamination has to be avoided. 

Can be done using statistics only for the training set

The mean, std, min, max and range of the training set will then be used to perform scaling for the test set as well. 

xxxxxxxxxxxxPerforming standardisation for the training set for both precip and temperaturexxxxx. Training will be performed on the bicubically interpolated dataset for now

In [22]:
from Standardise import standardise

Temperature 

Loading inputs and targets once again separately

#Scaling Precipitation training features and targets (from bicubic interpolation features): targets and features get scaled by their own mean and std. The same parametersa then used to standardise testing and validation

In [23]:
input_path = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Train/features_precip_masked_bicubic_train.nc"
output_path = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Train/SCALED_features_precip_masked_bicubic_train.nc"
var = "pr"
precip_min, precip_max = standardise(input_path, output_path, var)

/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/MyPythonEnvironment/lib/python3.12/site-packages/dask/_task_spec.py:758: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/MyPythonEnvironment/lib/python3.12/site-packages/dask/_task_spec.py:758: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/MyPythonEnvironment/lib/python3.12/site-packages/dask/_task_spec.py:758: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/MyPythonEnvironment/lib/python3.12/site-packages/dask/_task_spec.py:758: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)


Normalised/scaled variable saved to /work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Train/SCALED_features_precip_masked_bicubic_train.nc


In [12]:
input_path = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Train/targets_precip_masked_train.nc"
output_path = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Train/SCALED_targets_precip_masked_train.nc"
var = "RhiresD"
target_min, target_max = standardise(input_path, output_path, var)

Normalised/scaled variable saved to /work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Train/SCALED_targets_precip_masked_train.nc


Using above mean, max, std and min from temp and precip to standardise the vcalidation and testing data to avoid data leakage

xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxPRECIPITATIONxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

In [13]:
# Validation features :Precip
input_path = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Val/features_precip_masked_bicubic_val.nc"
output_path = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Val/SCALED_features_precip_masked_bicubic_val.nc"
var = "pr"
standardise(input_path, output_path, var, min=precip_min, max=precip_max)

/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/MyPythonEnvironment/lib/python3.12/site-packages/dask/_task_spec.py:758: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/MyPythonEnvironment/lib/python3.12/site-packages/dask/_task_spec.py:758: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/MyPythonEnvironment/lib/python3.12/site-packages/dask/_task_spec.py:758: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/MyPythonEnvironment/lib/python3.12/site-packages/dask/_task_spec.py:758: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)


Normalised/scaled variable saved to /work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Val/SCALED_features_precip_masked_bicubic_val.nc


(<xarray.DataArray 'pr' (N: 265, E: 370)> Size: 784kB
 dask.array<_nanmin_skip-aggregate, shape=(265, 370), dtype=float64, chunksize=(265, 370), chunktype=numpy.ndarray>
 Coordinates:
     lon      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
     lat      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
   * E        (E) float64 3kB 2.474e+06 2.476e+06 ... 2.842e+06 2.844e+06
   * N        (N) float64 2kB 1.06e+06 1.06e+06 1.062e+06 ... 1.322e+06 1.324e+06,
 <xarray.DataArray 'pr' (N: 265, E: 370)> Size: 784kB
 dask.array<_nanmax_skip-aggregate, shape=(265, 370), dtype=float64, chunksize=(265, 370), chunktype=numpy.ndarray>
 Coordinates:
     lon      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
     lat      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
   * E        (E) float64 3kB 2.474e+06 2.476e+06 ... 2.842e+06 2.844e+06
   * N        (N) float64 2kB 1.06e+06 1.06e+06 1.062e+06 ...

In [14]:

# Validation targets:  Precip
input_path = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Val/targets_precip_masked_val.nc"
output_path = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Val/SCALED_targets_precip_masked_val.nc"
var = "RhiresD"
standardise(input_path, output_path, var, min=target_min, max=target_max)

Normalised/scaled variable saved to /work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Val/SCALED_targets_precip_masked_val.nc


(<xarray.DataArray 'RhiresD' (N: 265, E: 370)> Size: 392kB
 dask.array<_nanmin_skip-aggregate, shape=(265, 370), dtype=float32, chunksize=(265, 370), chunktype=numpy.ndarray>
 Coordinates:
     lon      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
     lat      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
   * E        (E) float64 3kB 2.474e+06 2.476e+06 ... 2.842e+06 2.844e+06
   * N        (N) float64 2kB 1.06e+06 1.06e+06 1.062e+06 ... 1.322e+06 1.324e+06,
 <xarray.DataArray 'RhiresD' (N: 265, E: 370)> Size: 392kB
 dask.array<_nanmax_skip-aggregate, shape=(265, 370), dtype=float32, chunksize=(265, 370), chunktype=numpy.ndarray>
 Coordinates:
     lon      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
     lat      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
   * E        (E) float64 3kB 2.474e+06 2.476e+06 ... 2.842e+06 2.844e+06
   * N        (N) float64 2kB 1.06e+06 1.06e+06 1.0

In [15]:
# Precipitation test features
input_path = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Test/features_precip_masked_bicubic_test.nc"
output_path = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Test/SCALED_features_precip_masked_bicubic_test.nc"
var = "pr"
standardise(input_path, output_path, var, min=precip_min, max=precip_max)

/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/MyPythonEnvironment/lib/python3.12/site-packages/dask/_task_spec.py:758: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/MyPythonEnvironment/lib/python3.12/site-packages/dask/_task_spec.py:758: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/MyPythonEnvironment/lib/python3.12/site-packages/dask/_task_spec.py:758: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/MyPythonEnvironment/lib/python3.12/site-packages/dask/_task_spec.py:758: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)


Normalised/scaled variable saved to /work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Test/SCALED_features_precip_masked_bicubic_test.nc


(<xarray.DataArray 'pr' (N: 265, E: 370)> Size: 784kB
 dask.array<_nanmin_skip-aggregate, shape=(265, 370), dtype=float64, chunksize=(265, 370), chunktype=numpy.ndarray>
 Coordinates:
     lon      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
     lat      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
   * E        (E) float64 3kB 2.474e+06 2.476e+06 ... 2.842e+06 2.844e+06
   * N        (N) float64 2kB 1.06e+06 1.06e+06 1.062e+06 ... 1.322e+06 1.324e+06,
 <xarray.DataArray 'pr' (N: 265, E: 370)> Size: 784kB
 dask.array<_nanmax_skip-aggregate, shape=(265, 370), dtype=float64, chunksize=(265, 370), chunktype=numpy.ndarray>
 Coordinates:
     lon      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
     lat      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
   * E        (E) float64 3kB 2.474e+06 2.476e+06 ... 2.842e+06 2.844e+06
   * N        (N) float64 2kB 1.06e+06 1.06e+06 1.062e+06 ...

In [16]:
# Precipitation test targets
input_path = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Test/targets_precip_masked_test.nc"
output_path = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Test/SCALED_targets_precip_masked_test.nc"
var = "RhiresD"
standardise(input_path, output_path, var, min=target_min, max=target_max)

Normalised/scaled variable saved to /work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Test/SCALED_targets_precip_masked_test.nc


(<xarray.DataArray 'RhiresD' (N: 265, E: 370)> Size: 392kB
 dask.array<_nanmin_skip-aggregate, shape=(265, 370), dtype=float32, chunksize=(265, 370), chunktype=numpy.ndarray>
 Coordinates:
     lon      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
     lat      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
   * E        (E) float64 3kB 2.474e+06 2.476e+06 ... 2.842e+06 2.844e+06
   * N        (N) float64 2kB 1.06e+06 1.06e+06 1.062e+06 ... 1.322e+06 1.324e+06,
 <xarray.DataArray 'RhiresD' (N: 265, E: 370)> Size: 392kB
 dask.array<_nanmax_skip-aggregate, shape=(265, 370), dtype=float32, chunksize=(265, 370), chunktype=numpy.ndarray>
 Coordinates:
     lon      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
     lat      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
   * E        (E) float64 3kB 2.474e+06 2.476e+06 ... 2.842e+06 2.844e+06
   * N        (N) float64 2kB 1.06e+06 1.06e+06 1.0

xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxTEMPERATURE : Val and test targets and features standardisationxxxxxxxxxxxxxxxx


Training features 

In [17]:
tas_mean, tas_std = standardise(
    input_path="/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Train/features_tas_masked_bicubic_train.nc",
    output_path="/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Train/SCALED_features_tas_masked_bicubic_train.nc",
    var="tas"
)

/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/MyPythonEnvironment/lib/python3.12/site-packages/dask/_task_spec.py:758: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/MyPythonEnvironment/lib/python3.12/site-packages/dask/_task_spec.py:758: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/MyPythonEnvironment/lib/python3.12/site-packages/dask/_task_spec.py:758: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/MyPythonEnvironment/lib/python3.12/site-packages/dask/_task_spec.py:758: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)


Normalised/scaled variable saved to /work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Train/SCALED_features_tas_masked_bicubic_train.nc


Training targets 

In [18]:
tabsd_mean, tabsd_std = standardise(
    input_path="/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Train/targets_tas_masked_train.nc",
    output_path="/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Train/SCALED_targets_tas_masked_train.nc",
    var="TabsD"
)

/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/MyPythonEnvironment/lib/python3.12/site-packages/dask/array/numpy_compat.py:57: RuntimeWarning: invalid value encountered in divide
  x = np.divide(x1, x2, out)


Normalised/scaled variable saved to /work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Train/SCALED_targets_tas_masked_train.nc


Now we have the parameters from the training set for temperature, the same will be used to scale the featurea and targets of the Val and Test set

Validation standardisation

In [20]:
standardise(
    input_path="/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Val/features_tas_masked_bicubic_val.nc",
    output_path="/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Val/SCALED_features_tas_masked_bicubic_val.nc",
    var="tas",
    mean=tas_mean,
    std=tas_std
)

Normalised/scaled variable saved to /work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Val/SCALED_features_tas_masked_bicubic_val.nc


(<xarray.DataArray 'tas' (N: 265, E: 370)> Size: 784kB
 dask.array<mean_agg-aggregate, shape=(265, 370), dtype=float64, chunksize=(265, 370), chunktype=numpy.ndarray>
 Coordinates:
     lon      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
     lat      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
   * E        (E) float64 3kB 2.474e+06 2.476e+06 ... 2.842e+06 2.844e+06
   * N        (N) float64 2kB 1.06e+06 1.06e+06 1.062e+06 ... 1.322e+06 1.324e+06,
 <xarray.DataArray 'tas' (N: 265, E: 370)> Size: 784kB
 dask.array<_sqrt, shape=(265, 370), dtype=float64, chunksize=(265, 370), chunktype=numpy.ndarray>
 Coordinates:
     lon      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
     lat      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
   * E        (E) float64 3kB 2.474e+06 2.476e+06 ... 2.842e+06 2.844e+06
   * N        (N) float64 2kB 1.06e+06 1.06e+06 1.062e+06 ... 1.322e+06 1.324e+0

In [21]:
standardise(
    input_path="/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Val/targets_tas_masked_val.nc",
    output_path="/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Val/SCALED_targets_tas_masked_val.nc",
    var="TabsD",
    mean=tabsd_mean,
    std=tabsd_std
)

/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/MyPythonEnvironment/lib/python3.12/site-packages/dask/array/numpy_compat.py:57: RuntimeWarning: invalid value encountered in divide
  x = np.divide(x1, x2, out)


Normalised/scaled variable saved to /work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Val/SCALED_targets_tas_masked_val.nc


(<xarray.DataArray 'TabsD' (N: 265, E: 370)> Size: 392kB
 dask.array<mean_agg-aggregate, shape=(265, 370), dtype=float32, chunksize=(265, 370), chunktype=numpy.ndarray>
 Coordinates:
     lon      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
     lat      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
   * E        (E) float64 3kB 2.474e+06 2.476e+06 ... 2.842e+06 2.844e+06
   * N        (N) float64 2kB 1.06e+06 1.06e+06 1.062e+06 ... 1.322e+06 1.324e+06,
 <xarray.DataArray 'TabsD' (N: 265, E: 370)> Size: 392kB
 dask.array<_sqrt, shape=(265, 370), dtype=float32, chunksize=(265, 370), chunktype=numpy.ndarray>
 Coordinates:
     lon      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
     lat      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
   * E        (E) float64 3kB 2.474e+06 2.476e+06 ... 2.842e+06 2.844e+06
   * N        (N) float64 2kB 1.06e+06 1.06e+06 1.062e+06 ... 1.322e+06 1.32

Testing datasets for temepratures : standardisation of features and targets 

In [22]:
standardise(
    input_path="/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Test/features_tas_masked_bicubic_test.nc",
    output_path="/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Test/SCALED_features_tas_masked_bicubic_test.nc",
    var="tas",
    mean=tas_mean,
    std=tas_std
)

Normalised/scaled variable saved to /work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Test/SCALED_features_tas_masked_bicubic_test.nc


(<xarray.DataArray 'tas' (N: 265, E: 370)> Size: 784kB
 dask.array<mean_agg-aggregate, shape=(265, 370), dtype=float64, chunksize=(265, 370), chunktype=numpy.ndarray>
 Coordinates:
     lon      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
     lat      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
   * E        (E) float64 3kB 2.474e+06 2.476e+06 ... 2.842e+06 2.844e+06
   * N        (N) float64 2kB 1.06e+06 1.06e+06 1.062e+06 ... 1.322e+06 1.324e+06,
 <xarray.DataArray 'tas' (N: 265, E: 370)> Size: 784kB
 dask.array<_sqrt, shape=(265, 370), dtype=float64, chunksize=(265, 370), chunktype=numpy.ndarray>
 Coordinates:
     lon      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
     lat      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
   * E        (E) float64 3kB 2.474e+06 2.476e+06 ... 2.842e+06 2.844e+06
   * N        (N) float64 2kB 1.06e+06 1.06e+06 1.062e+06 ... 1.322e+06 1.324e+0

In [23]:
standardise(
    input_path="/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Test/targets_tas_masked_test.nc",
    output_path="/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Test/SCALED_targets_tas_masked_test.nc",
    var="TabsD",
    mean=tabsd_mean,
    std=tabsd_std
)

Normalised/scaled variable saved to /work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Test/SCALED_targets_tas_masked_test.nc


(<xarray.DataArray 'TabsD' (N: 265, E: 370)> Size: 392kB
 dask.array<mean_agg-aggregate, shape=(265, 370), dtype=float32, chunksize=(265, 370), chunktype=numpy.ndarray>
 Coordinates:
     lon      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
     lat      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
   * E        (E) float64 3kB 2.474e+06 2.476e+06 ... 2.842e+06 2.844e+06
   * N        (N) float64 2kB 1.06e+06 1.06e+06 1.062e+06 ... 1.322e+06 1.324e+06,
 <xarray.DataArray 'TabsD' (N: 265, E: 370)> Size: 392kB
 dask.array<_sqrt, shape=(265, 370), dtype=float32, chunksize=(265, 370), chunktype=numpy.ndarray>
 Coordinates:
     lon      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
     lat      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
   * E        (E) float64 3kB 2.474e+06 2.476e+06 ... 2.842e+06 2.844e+06
   * N        (N) float64 2kB 1.06e+06 1.06e+06 1.062e+06 ... 1.322e+06 1.32